In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

## Language modeling

### Data

In [2]:
# http://forums.fast.ai/t/where-to-download-data-for-these-lessons-4-5-arxiv-wikipedia-etc/8434/2
# Download data from https://einstein.ai/research/the-wikitext-long-term-dependency-language-modeling-dataset

PATH='data/wikitext-2/'
%ls {PATH}

models/  tmp/  wiki.test.tokens  wiki.train.tokens  wiki.valid.tokens


In [3]:
!head -5 {PATH}wiki.train.tokens

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [4]:
!wc -lwc {PATH}wiki.train.tokens

   36718  2051910 10797148 data/wikitext-2/wiki.train.tokens


In [5]:
!wc -lwc {PATH}wiki.valid.tokens

   3760  213886 1121681 data/wikitext-2/wiki.valid.tokens


In [6]:
TEXT = data.Field(lower=True)
FILES = dict(train='wiki.train.tokens', validation='wiki.valid.tokens', test='wiki.test.tokens')
bs,bptt = 80,70
#md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10) # This does not work.
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(365, 12980, 1, 2051911)

In [7]:
#md.trn_ds[0].text[:12], next(iter(md.trn_dl))

### Train

In [7]:
em_sz = 200
nh = 500
nl = 3
#learner = md.get_model(SGD_Momentum(0.7), bs, em_sz, nh, nl) # This does not work.
learner = md.get_model(SGD_Momentum(0.7), em_sz, nh, nl)
reg_fn=partial(seq2seq_reg, alpha=2, beta=1)

In [8]:
clip=0.3
#learner.fit(10, 1, wds=1e-6, reg_fn=reg_fn, clip=clip) # fit() got multiple values for keyword argument 'reg_fn'
learner.reg_fn = reg_fn
learner.clip = clip
learner.fit(10, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      6.154527   5.798778  



[5.7987776]

In [9]:
#learner.fit(10, 6, wds=1e-6, reg_fn=reg_fn, cycle_len=1, cycle_mult=2, clip=clip) # fit() got multiple values for keyword argument 'reg_fn'
#learner.fit(10, 6, wds=1e-6, cycle_len=1, cycle_mult=2) # 1 + 2 + 4 + 8 + 16 + 32 = 63
learner.fit(10, 1, wds=1e-6, cycle_len=1, cycle_mult=2) # quick

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      5.906548   5.597992  



[5.5979924]

In [10]:
learner.save('lm_420')

In [11]:
#learner.fit(10, 6, wds=1e-6, reg_fn=reg_fn, cycle_len=1, cycle_mult=2, clip=clip)
#learner.fit(10, 6, wds=1e-6, cycle_len=1, cycle_mult=2)
learner.fit(10, 1, wds=1e-6, cycle_len=1, cycle_mult=2) # quick

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      5.831246   5.486516  



[5.486516]

In [12]:
learner.save('lm_419')

In [13]:
#learner.fit(10, 6, wds=1e-6, reg_fn=reg_fn, cycle_len=1, cycle_mult=2, clip=clip)
#learner.fit(10, 6, wds=1e-6, cycle_len=1, cycle_mult=2)
learner.fit(10, 1, wds=1e-6, cycle_len=1, cycle_mult=2) # quick

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      5.755563   5.401954  



[5.401954]

In [14]:
learner.save('lm_418')

In [15]:
math.exp(4.17)

64.71545210740304

### Test

In [18]:
m=learner.model
s=[""". <eos> The game began development in 2010 , carrying over a large portion of the work 
done on Valkyria Chronicles II . While it retained the standard features of """.split()]
t=TEXT.numericalize(s) # torch.Size([30, 1])

m[0].bs=1 # batch size
#m.reset(False) # reset() takes 1 positional argument but 2 were given...
m.reset()
res,*_ = m(t) # res is torch.autograd.variable.Variable, torch.Size([30, 12980]), 12980 = number of tokens

In [26]:
nexts = torch.topk(res[-1], 10)[1] # returns 10 largest elements, torch.Size([10]), torch.topk() returns tuple(FloatTensor, LongTensor)
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['the', '<unk>', 'a', 'his', '"', 'an', 'their', 'its', 'her', 'this']

In [39]:
for i in range(20):
    n=res[-1].topk(2)[1] # res[-1].topk(2) returns ([torch.cuda.FloatTensor of size 2 (GPU 0)], [torch.cuda.LongTensor of size 2 (GPU 0)])
    n = n[1] if n.data[0]==0 else n[0] # TEXT.vocab.itos[0] == '<unk>'
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
    
# Example of unsqueeze()    
#>>> x = torch.Tensor([1, 2, 3, 4])
#>>> torch.unsqueeze(x, 0)
# 1  2  3  4

the first of the first @-@ year . the first of the first @-@ year , the first of the 

### End